In [1]:
import matplotlib.pyplot as plt
import numpy as np

import nengo
from nengo import spa

%load_ext nengo.ipynb
%matplotlib inline

# Improving function approximation with adjustment of tuning curves

This tutorial shows how adjusting the tuning curves of neurons can help to implement specific functions with Nengo. As an example we try to construct a simple cleanup for semantic pointers: Given an input vector ("stimulus"), the network should produce the sum of all semantic pointers which exceed a threshold similarity with the input vector.

Before you go through this tutorial, you should be familiar with the basics of Nengo and maybe have some idea of the Semantic Pointer Architecture (SPA).

We start by defining our vocabulary.

In [2]:
d = 64
vocab = spa.Vocabulary(d)
vocab.parse('A + B + C')

Our input will be $A + 0.2 B$ and we will use a threshold of 0.3. Thus, the cleaned up result should only be $A$.

In [3]:
stimulus_pointer = vocab['A'] + 0.2 * vocab['B']

## The standard approach

As a first pass we just use an ensemble with the default parameters and try to implement the cleanup function with the decoders being solved for.

In [4]:
def clean(x):
    threshold = 0.3
    similarities = np.dot(vocab.vectors, x)
    # `thresholded` will be 1 for all semantic pointers in the
    # vocabulary exceeding a similarity of 0.3 and 0 otherwise.
    thresholded = np.maximum(0, np.sign(similarities - threshold))
    # sum all semantic pointers exceeding the threshold
    return np.dot(thresholded, vocab.vectors)

with nengo.Network() as cleanup:
    stimulus = nengo.Node(stimulus_pointer.v)
    ens = nengo.Ensemble(n_neurons=150, dimensions=d)
    output = nengo.Node(size_in=d)
    
    nengo.Connection(stimulus, ens)
    nengo.Connection(ens, output, function=clean)
    
    p = nengo.Probe(output, synapse=0.005)

In [5]:
with nengo.Simulator(cleanup) as sim:
    sim.run(0.2)

In [6]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], vocab))
plt.legend(vocab.keys)
plt.xlabel("t")
plt.ylabel("Similarity")

We see that this approach does not work very well. The curves are noisy and even though it is most similar to $A$, this similarity is considerably below 1.

## Using an ensemble array

In the first try we tried to implement the dot product between the input and all vocabulary items with the decoders. We can introduce a transform to do this and then the decoders need to implement only the thresholding. That is a much simpler function.

Step by step, the following will happen: The stimulus vector gets multiplied with a transform matrix consisting of all the semantic pointers in the vocabulary. This yields a vector of similarities between the stimulus vector and the semantic pointers in the vocabulary. Each element in this vector will be thresholded and then multiplied with the transpose of the initial transform matrix. This converts the thresholded scalar similaritiesback into the semantic pointers and adds them up.

Because all of the similarities are independent, we will use an ensemble array to represent these values.

In [7]:
with nengo.Network() as cleanup:
    stimulus = nengo.Node(stimulus_pointer.v)
    ea = nengo.networks.EnsembleArray(
        n_neurons=50, n_ensembles=len(vocab.vectors), ens_dimensions=1)
    output = nengo.Node(size_in=d)
    
    # This connection determins the similarities with the transform.
    nengo.Connection(stimulus, ea.input, transform=vocab.vectors)
    # Decoders from the ensemble array's ensembles do the thresholding.
    ea.add_output('threshold', lambda x: 0. if x < 0.3 else 1.)
    # This connections transforms the thresholded values back to semantic pointers.
    nengo.Connection(ea.threshold, output, transform=vocab.vectors.T)
    
    p = nengo.Probe(output, synapse=0.005)

In [8]:
with nengo.Simulator(cleanup) as sim:
    sim.run(0.2)

In [9]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], vocab))
plt.legend(vocab.keys)
plt.xlabel("t")
plt.ylabel("Similarity")

This is a lot better. The similarity of the output with $A$ is now close to 1 and there is much less noise. But we can improve this even further. Especially the similarity with the $B$ pointer is above 0 despite the thresholding.

## Investigating the tuning curves

Let us take a look at the tuning curves of the neurons in one of the thresholding ensembles.

In [10]:
plt.plot(*nengo.utils.ensemble.tuning_curves(ea.ensembles[0], sim))
plt.xlabel("Input")
plt.ylabel("Firing rate [Hz]")

About half of these neurons are tuned to fire more for smaller values. But these values are not really relevant as we are doing a thresholding at 0.3. Thus, we change all neurons to be tuned to fire more for larger values by setting all the encoders to be positive.

In [11]:
with nengo.Network() as cleanup:
    stimulus = nengo.Node(stimulus_pointer.v)
    ea = nengo.networks.EnsembleArray(
        n_neurons=50, n_ensembles=len(vocab.vectors), ens_dimensions=1,
        encoders=nengo.dists.Choice([[1]]))
    output = nengo.Node(size_in=d)
    
    nengo.Connection(stimulus, ea.input, transform=vocab.vectors)
    ea.add_output('threshold', lambda x: 0. if x < 0.3 else 1.)
    nengo.Connection(ea.threshold, output, transform=vocab.vectors.T)
    
    p = nengo.Probe(output, synapse=0.005)

In [12]:
with nengo.Simulator(cleanup) as sim:
    sim.run(0.2)

The resulting tuning curves:

In [13]:
plt.plot(*nengo.utils.ensemble.tuning_curves(ea.ensembles[0], sim))
plt.xlabel("Input")
plt.ylabel("Firing rate [Hz]")

In [15]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], vocab))
plt.legend(vocab.keys)
plt.xlabel("t")
plt.ylabel("Similarity")

Compared to the previous plot the similarity with the $B$ and $C$ pointer is less noisy. The tuning curves are now all aligned in the correct direction, but they are still covering a lot of irrelevant area. Because all values below 0.3 should be 0, there is no need to have neurons tuned to this range. We want to shift all the intercepts to the range $(0.3, 1.0)$.

But not only the range of intercepts can be important, but also the distribution of intercepts. Let us take a look at the thresholding function:

In [16]:
xs = np.linspace(-1, 1, 100)
plt.plot(xs, xs >= 0.3)
plt.ylim(-0.1, 1.1)

This function is mostly constant except for that large jump at the threshold. The constant parts are easy to approximate and will not need a lot of neural resources, but the highly non-linear jump will require much more neural resources for an accurate representation.

Thus, let us try to implement just this thresholding of a one-dimensional scalar in three ways: With a uniform distribution of intercepts (the default), all intercepts at 0.3 (where we have the non-linearity), and an exponential distribution. The last approach is in between the two extremes of a uniform distribution and placing all intercepts at 0.3. It will distribute most intercepts close to 0.3, but some intercepts will still be at larger values.

In [17]:
threshold = 0.3
with nengo.Network() as threshold_net:
    stimulus = nengo.Node(lambda t: t)
    ens_uniform = nengo.Ensemble(
        n_neurons=50, dimensions=1,
        encoders=nengo.dists.Choice([[1]]), intercepts=nengo.dists.Uniform(threshold, 1.))
    ens_fixed = nengo.Ensemble(
        n_neurons=50, dimensions=1,
        encoders=nengo.dists.Choice([[1]]), intercepts=nengo.dists.Choice([threshold]))
    ens_exp = nengo.Ensemble(
        n_neurons=50, dimensions=1,
        encoders=nengo.dists.Choice([[1]]), intercepts=nengo.dists.Exponential(0.15, threshold, 1.))
    
    out_uniform = nengo.Node(size_in=1)
    out_fixed = nengo.Node(size_in=1)
    out_exp = nengo.Node(size_in=1)
    
    threshold_fn = lambda x: 0. if x < threshold else 1.
    nengo.Connection(stimulus, ens_uniform)
    nengo.Connection(stimulus, ens_fixed)
    nengo.Connection(stimulus, ens_exp)
    nengo.Connection(ens_uniform, out_uniform, function=threshold_fn)
    nengo.Connection(ens_fixed, out_fixed, function=threshold_fn)
    nengo.Connection(ens_exp, out_exp, function=threshold_fn)
    
    p_uniform = nengo.Probe(out_uniform, synapse=0.005)
    p_fixed = nengo.Probe(out_fixed, synapse=0.005)
    p_exp = nengo.Probe(out_exp, synapse=0.005)

In [18]:
with nengo.Simulator(threshold_net) as sim:
    sim.run(1.)

Let as look at the tuning curves first.

In [19]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(*nengo.utils.ensemble.tuning_curves(ens_uniform, sim))
plt.xlabel("Input")
plt.ylabel("Firing rate [Hz]")
plt.title("Uniform intercepts")

plt.subplot(1, 3, 2)
plt.plot(*nengo.utils.ensemble.tuning_curves(ens_fixed, sim))
plt.xlabel("Input")
plt.ylabel("Firing rate [Hz]")
plt.title("Fixed intercepts")

plt.subplot(1, 3, 3)
plt.plot(*nengo.utils.ensemble.tuning_curves(ens_exp, sim))
plt.xlabel("Input")
plt.ylabel("Firing rate [Hz]")
plt.title("Exponential intercept distribution")

Now let us look at how these three ensembles approximate the thresholding function.

In [20]:
plt.plot(sim.trange(), sim.data[p_uniform], label='Uniform intercepts')
plt.plot(sim.trange(), sim.data[p_fixed], label='Fixed intercepts')
plt.plot(sim.trange(), sim.data[p_exp], label='Exponential intercept dist.')
plt.xlabel('Input')
plt.ylabel('Output')
plt.legend(loc='best')

We see that the fixed intercepts produce slightly higher decoded values close to the threshold, but the slope is lower than for uniform intercepts. The best approximation of the thresholding is done with the exponential intercept distribution. Here we get a quick rise to 1 at the threshold and a fairly constant representation of 1 for value sufficiently above the threshold. Thus, we will use this intercept distribution in our final cleanup network.

Nengo provides the `ThresholdingEnsemble` preset to make it easier to assign intercepts according to that distribution as well as adjusting the encoders and evaluation points accordingly.

In [21]:
with nengo.Network() as cleanup:
    stimulus = nengo.Node(stimulus_pointer.v)
    with nengo.presets.ThresholdingEnsembles(0.3):
        ea = nengo.networks.EnsembleArray(n_neurons=50, n_ensembles=len(vocab.vectors))
    output = nengo.Node(size_in=d)
    
    nengo.Connection(stimulus, ea.input, transform=vocab.vectors)
    ea.add_output('threshold', lambda x: 0. if x < 0.3 else 1.)
    nengo.Connection(ea.threshold, output, transform=vocab.vectors.T)
    
    p = nengo.Probe(output, synapse=0.005)

In [22]:
with nengo.Simulator(cleanup) as sim:
    sim.run(0.2)

In [23]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], vocab))
plt.legend(vocab.keys)
plt.xlabel("t")
plt.ylabel("Similarity")

This gives the best cleanup with the least amount of noise.

The take-away from this tutorial is that adjusting ensemble parameters in the right way can sometimes help in implementing functions more acurately in neurons.